$\Large\textbf{Lab 4.}$ $\large\textbf{Exercise 2.}$



Consider the problem $\min_{\mathbf{x}} q(\mathbf{x}) = \sqrt{x_1^2 + 4} +  \sqrt{x_2^2 + 4}$. 

Hessian of $ q(\mathbf{x})$ is shown below.

$q(\mathbf{x}) = \sqrt{x_1^2 + 4} +  \sqrt{x_2^2 + 4}$

$\therefore\nabla^2 q(\mathbf{x}) = 
\begin{bmatrix}
  q_{x_1^2}(\mathbf{x}) & 
    q_{x_1x_2}(\mathbf{x})  \\
  q_{x_2x_1}(\mathbf{x}) & 
    q_{x_2^2}(\mathbf{x})
\end{bmatrix} = 
\begin{bmatrix}
  \frac{1}{(x_1^2 + 4)^{\frac{3}{2}}} & 0 \\ 0 & \frac{1}{(x_2^2 + 4)^{\frac{3}{2}}}
\end{bmatrix}$  \\

In [ ]:
import numpy as np 
import scipy
from scipy.linalg import sqrtm
#method to find Hessian matrix
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  a11 = (4 + x[0]**2)**(-3/2)
  a12 = 0
  a21 = 0
  a22 = (4 + x[1]**2)**(-3/2)
  a = np.array([[a11, a12], [a21, a22]])
  return a

The function $ q(\mathbf{x})$ and it's gradient $\nabla q(\mathbf{x})$

In [ ]:
#Now we will define a function which will compute and return the function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return (4 + x[0]**2)**0.5 + (4 + x[1]**2)**0.5

In [ ]:
#Now we will define a function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  a = (x[0]) / (4 + x[0]**2)**(0.5)
  b = (x[1]) / (4 + x[1]**2)**(0.5)
  return np.array([a, b])

**Newton's Method**

$\mathbf{D}^k = (\nabla^2 q(\mathbf{x}^k))^{-1}$

In [ ]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  if np.linalg.det(evalh(x)) == 0:
    print("Determinant of Hessian matrix is:", np.linalg.det(evalh(x)), "When hessian is:\n", evalh(x))
    raise ValueError('Determinant is zero of the given matrix therefore inverse does not exists please select different initial point!')
  #print(evalh(x))
  return np.linalg.inv(evalh(x))
  #compute and return D_k


In [ ]:
#line search type 
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [ ]:
def compute_steplength_backtracking_scaled_direction_newton(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  D_k = compute_D_k(x)
  #implement the backtracking line search
  #print(evalf(x + alpha*p), evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(p), p) ))
  while evalf(x + alpha*np.matmul(D_k,p)) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha  

In [ ]:
#complete the code for gradient descent with scaling to find the minimizer
def find_minimizer_gdscaling_newton(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    D_k = compute_D_k(x)
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction_newton(x, g_x, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:  
      raise ValueError('Line search type unknown. Please check!')
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
  return x, k
  #Complete the code   

$\textbf{Question 2 and 4:}$


$\textbf{Question 2:}$
We consider $\eta^k = 1, \forall k = 1, 2, ...$ in Algorithm 1. With starting point $\mathbf{x}^0 = (2, 2)$ and a stopping tolerance
$\tau = 10^{-9}$


In [ ]:
my_start_x = np.array([2.,2.])
my_tol= 1e-9

#check gradient descent with scaling and backtracking line search 
alpha_start = 1.
rho = 0.5
gamma = 0.5

print("\nUsing Newton's method with constant step length = 1:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gdscaling_newton(my_start_x, my_tol, CONSTANT_STEP_LENGTH, alpha_start, rho, gamma)
print("Minimizer: {}, \nMinimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)


print("\nUsing Newton's method with scaling and Backtracking Line Search for step length:")
#check what happens when you call find_minimzer using backtracking line search
x_opt, k = find_minimizer_gdscaling_newton(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, \nMinimum function value: {}".format((x_opt[0], x_opt[1]), evalf(x_opt)),"\nNo. of Iterations:", k)


Using Newton's method with constant step length = 1:
Determinant of Hessian matrix is: 0.0 When hessian is:
 [[9.04743327e-280 0.00000000e+000]
 [0.00000000e+000 9.04743327e-280]]


ValueError: ignored


**Newton's Method with  $\eta^k = 1$ as a constant step length:**

Minimizer: $(3.933899782532571e-12, 3.933899782532571e-12)$

Minimum function value: $4.0 $

No. of Iterations: $38$

**Newton's Method (with backtracking line search):**

Minimizer: $(2.220446049250313e-16, 2.220446049250313e-16)$

Minimum function value: $4.0 $

No. of Iterations: $1$

We see that number of iterations to converge the newton's method with $\eta^k = 1$ are much higher than the newton's method with backtracking line search because backtracking line search is finding $\eta^k$ at each iterations using scaled algorithm of backtracking line search which is improving the condition number of a matrix $\mathbf{Q}$. Here,

$\mathbf{Q}$ = $(\mathbf{D}^k)^{\frac{1}{2}} \nabla^2q(\mathbf{x}) (\mathbf{D}^k)^{\frac{1}{2}} = (\mathbf{D}^k)^{\frac{1}{2}} \mathbf{H}^k (\mathbf{D}^k)^{\frac{1}{2}}$

where, $\mathbf{D}^k = (\nabla^2q(\mathbf{x}))^{-1} $

Hence backtracking algorithm is taking just one iteration to converge where as newton's method with constant step length is taking $38$ iterations. Since $(2,2)$ is much closer to optimizer, the method is converging.


$\textbf{Question 4:}$

We consider $\eta^k = 1, \forall k = 1, 2, ...$ in Algorithm 1. With starting point $\mathbf{x}^0 = (8, 8)$ and a stopping tolerance
$\tau = 10^{-9}$

In [ ]:
my_start_x = np.array([8.,8.])
my_tol= 1e-9

#check gradient descent with scaling and backtracking line search 
alpha_start = 1.
rho = 0.5
gamma = 0.5

print("\nUsing Newton's method with constant step length = 1:")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gdscaling_newton(my_start_x, my_tol, CONSTANT_STEP_LENGTH, alpha_start, rho, gamma)
print("Minimizer: {}, \nMinimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)



Using Newton's method with constant step length = 1:
Determinant of Hessian matrix is: 0.0 When hessian is:
 [[8.66023328e-222 0.00000000e+000]
 [0.00000000e+000 8.66023328e-222]]


ValueError: ignored

Here in this method, We see that in the newton's method with $\eta^k = 1$, algorithm is not converging because whenever we are trying to find scaling matrix $\mathbf{D}^k$, in the $5^{th}$ iteration, the hessian matrix becomes singular because of that we cannot find proper scaling factor and algorithm is giving error. If we choose different initial point then algorithm might converge. If we choose exact line search or backtracking line search then also we might converge but since we are keeping step length constant, algorithm is diverging.

Newton's method converges for quadratic function within one step while for higher degree functions, it is taking quadratic approximation of the given function in some neighbourhood of agiven initial point and then finds minimum around that quadratic approoximation. Newton's method does not converges globally for each function. If our initial point is not close to optimum then newton's method might not converge or it's hessian might become singular for some functions and therefore method will fail to find optimum. In this particular example, $(10,10)$ is not close to optimum $(0,0)$ therefore method is failing to converge. Because of constant step length and $(8,8)$ is not close to optimum $(0,0)$, the algorithm is moving in ascent direction and diverges.

In [ ]:
my_start_x = np.array([8.,8.])
my_tol= 1e-9

#check gradient descent with scaling and backtracking line search 
alpha_start = 1.
rho = 0.5
gamma = 0.5

print("\nUsing Newton's method with scaling and Backtracking Line Search for step length:")
#check what happens when you call find_minimzer using backtracking line search
x_opt, k = find_minimizer_gdscaling_newton(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, \nMinimum function value: {}".format((x_opt[0], x_opt[1]), evalf(x_opt)),"\nNo. of Iterations:", k)

Here we see that the algorithm using Newton's method with scaling and Backtracking Line Search for finding the step length is converging within $17$ iterations. Since step length is not constant, algorithm is taking descent direction in each iteration and fially converging within $17$ steps. Here hessian is not becoming singular since we wre moving in the descent direction using backtracking line search. In the constant step length we were moving in the ascent direction because of constant step length $\eta^k = 1$.

$\textbf{Question 3 and 5:}$

Using diagonal matrix

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k_diagonal(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  A = evalh(x)
  B = np.array([[1/A[0][0], 0], [0, 1/A[1][1]]])
  return B

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  #implement the backtracking line search
  #print(evalf(x + alpha*p), evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(p), p) ))
  while evalf(x + alpha*p) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), p) ):
    alpha = alpha * rho
  return alpha
  #Complete the code 

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  #assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  D_k = compute_D_k_diagonal(x)
  #implement the backtracking line search
  #print(evalf(x + alpha*p), evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(p), p) ))
  while evalf(x + alpha*np.matmul(D_k,p)) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha  
  
  #Complete the code 

In [ ]:
#complete the code for gradient descent to find the minimizer
def find_minimizer_gd(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  A = evalh(x)/2
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x, g_x, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length, g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
  return x, k
  #Complete the code   


In [ ]:
#complete the code for gradient descent with scaling to find the minimizer

def find_minimizer_gdscaling(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  #A = evalh(x)/2
  
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    D_k = compute_D_k_diagonal(x)
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
  return x, k
  #Complete the code   

$\textbf{Question 3:}$
We consider $\alpha^0 = 1, \rho = 0.5, \gamma = 0.5$ in Algorithm 1. With starting point $\mathbf{x}^0 = (2, 2)$ and a stopping tolerance
$\tau = 10^{-9}$

In [ ]:
my_start_x = np.array([2.,2.])
my_tol= 1e-9

#check gradient descent with backtracking line search 
alpha_start = 1.
rho = 0.5
gamma = 0.5
print("\nUsing Gradient descent with Backtracking Line Search")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)

print("\nUsing Gradient descent with scaling and Backtracking Line Search")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)


Using Gradient descent with Backtracking Line Search
Minimizer: (7.625256379536198e-10, 7.625256379536198e-10), Minimum function value: 4.0 
No. of Iterations: 32

Using Gradient descent with scaling and Backtracking Line Search
Minimizer: (4.440892098500626e-16, 4.440892098500626e-16), Minimum function value: 4.0 
No. of Iterations: 1


**Newton's Method with  $\eta^k = 1$ as a constant step length:**

Minimizer: $(2.83764946761543e-12, 2.83764946761543e-12)$

Minimum function value: $4.0 $

No. of Iterations: $13$

**Gradient descent algorithm (with scaling) with backtracking line search using diagonal matrix as an approximations of hessian inverse:**

Minimizer: $(4.440892098500626e-16, 4.440892098500626e-16)$

Minimum function value: $4.0 $

No. of Iterations: $1$

**Gradient descent algorithm (without scaling) with backtracking line search using diagonal matrix as an approximations of hessian inverse:**

Minimizer: $(7.625256379536198e-10, 7.625256379536198e-10)$

Minimum function value: $4.0 $

No. of Iterations: $32$

We can see that hessian matrix is diagonal so Newton's method and Gradient descent algorithm with backtracking line search using diagonal matrix as an approximations of hessian inverse are doing exactly same thing.


We see that Newton's Method with  $\eta^k = 1$ as a constant step length is taking $13$ iterations which are much higher than Gradient descent algorithm with backtracking line search using diagonal matrix as an approximations of hessian inverse (with scaling) which is taking just one iteration to converge. Here, because of scaling and backtracking, algorithm is converging so quickly. Our Initial point $(2,2)$ is also close to optimum $(0,0)$ because of that function can be approximated well in quadratic form around the initial point therefore algorithm is converging within one step. On the other hand, because of the constant step length, $13$ iterations are needed.

When we use Gradient descent algorithm (without scaling) with backtracking line search using diagonal matrix as an approximations of hessian inverse, algorithm is converging within $32$ steps. Here we are not scaling because of that backtracking line search is taking some more time to converge as compared to scaled algorithm with backtracking line search. Constant step length is least favorable because step length is not calculated at each iteration because of that, maximum $32$ iterations are needed. Therefore for this function, most prefferd algorithm is backtracking with scaling with initial point as $(2,2)$.

$\textbf{Question 5:}$
We consider $\alpha^0 = 1, \rho = 0.5, \gamma = 0.5$ in Algorithm 1. With starting point $\mathbf{x}^0 = (8, 8)$ and a stopping tolerance
$\tau = 10^{-9}$

In [ ]:
my_start_x = np.array([8.,8.])
my_tol= 1e-9

#check gradient descent with backtracking line search 
alpha_start = 1.
rho = 0.5
gamma = 0.5
print("\nUsing Gradient descent with Backtracking Line Search without scaling using diagonal matrix as an approximations of hessian inverse")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)

print("\nUsing Gradient descent with Backtracking Line Search with scaling using diagonal matrix as an approximations of hessian inverse")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1]), evalf(x_opt_bls)),"\nNo. of Iterations:", k)


Using Gradient descent with Backtracking Line Search without scaling using diagonal matrix as an approximations of hessian inverse
Minimizer: (8.317704696930586e-10, 8.317704696930586e-10), Minimum function value: 4.0 
No. of Iterations: 39

Using Gradient descent with Backtracking Line Search with scaling using diagonal matrix as an approximations of hessian inverse
Minimizer: (2.83764946761543e-12, 2.83764946761543e-12), Minimum function value: 4.0 
No. of Iterations: 13


**Newton's Method with  $\eta^k = 1$ as a constant step length:**

Method is not converging because of singular hessian in the iterations.


**Gradient descent with Backtracking Line Search without scaling using diagonal matrix as an approximations of hessian inverse**

Minimizer: $(8.317704696930586e-10, 8.317704696930586e-10)$

Minimum function value: $4.0 $

No. of Iterations: $39$

**Gradient descent with Backtracking Line Search with scaling using diagonal matrix as an approximations of hessian inverse**

Minimizer: $(2.83764946761543e-12, 2.83764946761543e-12)$

Minimum function value: $4.0 $

No. of Iterations: $ 13$

We can see that hessian matrix is diagonal so Newton's method and Gradient descent algorithm with backtracking line search using diagonal matrix as an approximations of hessian inverse are doing exactly same thing.


We see that Newton's Method with  $\eta^k = 1$ as a constant step length is not converging while the Gradient descent algorithm with backtracking line search using diagonal matrix as an approximations of hessian inverse (with scaling) which is taking just $13$ iterations to converge. Here, because of scaling and backtracking, algorithm is converging. Our Initial point $(8,8)$ is also not close to optimum $(0,0)$ because of that function cannot be approximated well in quadratic form around the initial point therefore algorithm is converging within $13$ step. On the other hand, because of the constant step length, algorithm is not converging.

When we use Gradient descent algorithm (without scaling) with backtracking line search using diagonal matrix as an approximations of hessian inverse, algorithm is converging within $39$ steps. Here we are not scaling still backtracking line search is not taking more time to converge as compared to scaled algorithm with backtracking line search. Constant step length is not favorable because step length is not calculated at each iteration because of that, algorithm is not converging and diverging to infinity. Therefore for this initial point $(8,8)$, most prefferd algorithm is backtracking without scaling and then same algorithm with scaling.